In [1]:
#Importing the data
import pandas as pd
import numpy as np
import pandas as pd
from surprise import Reader, Dataset, KNNBasic, accuracy, PredictionImpossible
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from collections import defaultdict
#Importing the data
data = pd.read_csv('/home/bbruno/all_here/python course/vinnie/data/cleaned_data/merged_df.csv')
#use cols userId, wine, rating
data = data[['userId', 'wine', 'rate']]
data.head()

,userId,wine,rate
0,826c6abe-7147-4b69-aaea-275fe65b033a,Cavalcaonte Barbera Forlí,5
1,d838a526-4b0a-4fad-b54f-90b6ca321fca,Shiraz,2
2,d838a526-4b0a-4fad-b54f-90b6ca321fca,Pinot Noir,3
3,bec96c0b-5b0b-4992-8756-e0d3b704afb5,Timorasso,5
4,bec96c0b-5b0b-4992-8756-e0d3b704afb5,Le Difese,4


In [2]:
class Knn (KNNBasic):
    def __init__(self, sim_options={}, bsl_options={}):
        KNNBasic.__init__(self, sim_options=sim_options, bsl_options=bsl_options)
    
    def create_reader(self, data):
        reader = Reader(rating_scale=(1, 5))
        self.data = Dataset.load_from_df(data[['userId', 'wine', 'rate']], reader)
    # option 1
    ########################################
    # for cross validation we have two functions, cross_validate and fit
    def cross_validate(self, measures=['RMSE'], cv=3, verbose=False):
        results = cross_validate(self, self.data, measures=measures, cv=cv, verbose=verbose)
        for measure in measures:
            print(f'{measure}: {results["test_" + measure.lower()].mean()}')
        return results
    
    def fit(self, trainset):
        predictions = KNNBasic.fit(self, trainset).test(trainset.build_testset())
        self.sim = self.compute_similarities()
        self.bu, self.bi = self.compute_baselines()
        return predictions
    ########################################
    # option 2
    ########################################
    # # fit funtion that works without cross validation
    # def fit (self):
    #     self.trainset, testset = train_test_split(self.data, test_size=0.2)
    #     predictions = KNNBasic.fit(self, self.trainset).test(testset)
    #     self.sim = self.compute_similarities()
    #     self.bu, self.bi = self.compute_baselines()
    #     return predictions
    ########################################

    def estimated(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible("User and/or item is unknown.")
        
        # Compute similarities between u and v, where v describes all other
        # users that have also rated item i.
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Sort these neighbors by similarity
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print("The 5 nearest neighbors of user", str(u), "are:")
        for v, sim_uv in neighbors[:5]:
            print(f"user {v} with sim {sim_uv:1.15f}")

        # ... Aaaaand return the baseline estimate anyway ;)
        bsl = self.trainset.global_mean + self.bu[u] + self.bi[i]
        return print(f"And the baseline estimate is: {bsl}")
    
    def get_Iu(self, uid):
        """Return the number of items rated by given user
        args:
          uid: the id of the user
        returns:
          the number of items rated by the user
        """
        try:
            return len(self.trainset.ur[self.trainset.to_inner_uid(uid)])
        except ValueError:  # user was not part of the trainset
            return 0

    def get_Ui(self, iid):
        """Return the number of users that have rated given item
        args:
          iid: the raw id of the item
        returns:
          the number of users that have rated the item.
        """
        try:
            return len(self.trainset.ir[self.trainset.to_inner_iid(iid)])
        except ValueError:
            return 0

    def inspect_predictions(self, predictions):
        print(f"uid means the user id and iid means the wine id\n")
        print(f"rui means the actual rating and est means the estimated rating\n")
        print(f"err means the error between the actual and the estimated rating\n")
        print(f"Iu means the number of items rated by given user\n")
        print(f"Ui means the number of users that have rated given item\n")
        # Create a dataframe with the predictions
        df_pred = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
        df_pred['Iu'] = df_pred.uid.apply(self.get_Iu)
        df_pred['Ui'] = df_pred.iid.apply(self.get_Ui)
        df_pred['err'] = abs(df_pred.est - df_pred.rui)
        return df_pred
    
    def get_accuracy(self, predictions, k=10, threshold=3.5):
        # Compute RMSE
        accuracy.rmse(predictions, verbose=True)
        
        # Compute precision and recall
        precisions, recalls = self.precision_recall_at_k(predictions, k=k, threshold=threshold)

        # Precision and recall can then be averaged over all users
        precision = sum(prec for prec in precisions.values()) / len(precisions)
        recall = sum(rec for rec in recalls.values()) / len(recalls)
        print(f'Precision: {precision:.2f}\nRecall: {recall:.2f}')

        # Count correct predictions
        correct = 0
        for uid, iid, true_r, est, _ in predictions:
            if round(est) == round(true_r):
                correct += 1

        # Compute accuracy
        accuracy_percentage = correct / len(predictions)
        return accuracy_percentage * 100
    
    @staticmethod 
    def precision_recall_at_k(predictions, k=10, threshold=3.5):
        """Return precision and recall at k metrics for each user"""

        # First map the predictions to each user.
        user_est_true = defaultdict(list)
        for uid, _, true_r, est, _ in predictions:
            user_est_true[uid].append((est, true_r))

        precisions = dict()
        recalls = dict()
        for uid, user_ratings in user_est_true.items():

            # Sort user ratings by estimated value
            user_ratings.sort(key=lambda x: x[0], reverse=True)

            # Number of relevant items
            n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

            # Number of recommended items in top k
            n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

            # Number of relevant and recommended items in top k
            n_rel_and_rec_k = sum(
                ((true_r >= threshold) and (est >= threshold))
                for (est, true_r) in user_ratings[:k]
            )

            # Precision@K: Proportion of recommended items that are relevant
            # When n_rec_k is 0, Precision is undefined. We here set it to 0.

            precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

            # Recall@K: Proportion of relevant items that are recommended
            # When n_rel is 0, Recall is undefined. We here set it to 0.

            recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

        return precisions, recalls
    
    # Mi function acc that works perfectly was modifyed to add precision and recall, for that reason this is commented
    # def get_accuracy(self, predictions):
    #     # # Compute RMSE
    #     # predictions = KNNBasic.test(self, testset)
    #     accuracy.rmse(predictions, verbose=True)
    #     # Count correct predictions
    #     correct = 0
    #     for uid, iid, true_r, est, _ in predictions:
    #         if round(est) == round(true_r):
    #             correct += 1

    #     # Compute accuracy
    #     accuracy_percentage = correct / len(predictions)
    #     return accuracy_percentage * 100

Here's a brief explanation of each step:

1. `knn.create_reader(data)`: This loads your data into the KNN model.
2. `knn.cross_validate(cv=3)`: This performs cross-validation on your data with 3 folds. It's good to do this before training your model to get an idea of how well it might perform.
3. `trainset = knn.data.build_full_trainset()`: This builds the full trainset from your data.
4. `predictions = knn.fit(trainset)`: This trains your model on the full trainset and generates predictions.
5. `knn.estimated(140, 10)`: This estimates the rating that user 140 would give to item 10.
6. `df_pred = knn.inspect_predictions(predictions)`: This inspects your predictions.
7. `df_pred.head(10)`: This displays the first 10 rows of your predictions.

For your next steps, you might want to:
- Check the accuracy of your model
- Try different values for `cv` to see how it affects your model's performance
- Experiment with different parameters for your KNN model to see if you can improve its performance.

In [3]:
knn = Knn(
    sim_options = {'name': 'pearson_baseline','user_based': True}, 
    bsl_options={'method': 'sgd', 'learning_rate': 0.00005, 'n_epochs':20, 'reg_u': 12 , 'reg_i': 5}
    )

In [4]:
knn.create_reader(data)

In [5]:
knn.cross_validate(cv=3)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9093090782925309


{'test_rmse': array([0.90707399, 0.90473983, 0.91611341]),
 'fit_time': (0.015426158905029297,
  0.0066547393798828125,
  0.0047054290771484375),
 'test_time': (0.0018405914306640625,
  0.0011758804321289062,
  0.001482248306274414)}

In [6]:
trainset = knn.data.build_full_trainset()
predictions = knn.fit(trainset)
# knn.cross_validate(knn.data)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [7]:
# predictions = knn.fit()

In [8]:
knn.estimated(147, 355)
print("If the baseline is {} then the value is a default value".format(knn.trainset.global_mean))

The 5 nearest neighbors of user 147 are:
user 87 with sim 0.000000000000000
And the baseline estimate is: 3.599788448536674
If the baseline is 3.6011904761904763 then the value is a default value


In [9]:
df_pred = knn.inspect_predictions(predictions)
best_pred = df_pred.sort_values(by='err')[:10]
worst_pred = df_pred.sort_values(by='err')[-10:]
df_pred.head(10)

uid means the user id and iid means the wine id

rui means the actual rating and est means the estimated rating

err means the error between the actual and the estimated rating

Iu means the number of items rated by given user

Ui means the number of users that have rated given item



,uid,iid,rui,est,details,Iu,Ui,err
0,826c6abe-7147-4b69-aaea-275fe65b033a,Cavalcaonte Barbera Forlí,5.0,5.000000,"{'actual_k': 1, 'was_impossible': False}",1,1,0.000000
1,d838a526-4b0a-4fad-b54f-90b6ca321fca,Shiraz,2.0,2.000000,"{'actual_k': 1, 'was_impossible': False}",2,2,0.000000
2,d838a526-4b0a-4fad-b54f-90b6ca321fca,Pinot Noir,3.0,3.001229,"{'actual_k': 3, 'was_impossible': False}",2,7,0.001229
3,bec96c0b-5b0b-4992-8756-e0d3b704afb5,Timorasso,5.0,5.000000,"{'actual_k': 1, 'was_impossible': False}",6,1,0.000000
4,bec96c0b-5b0b-4992-8756-e0d3b704afb5,Le Difese,4.0,4.000000,"{'actual_k': 1, 'was_impossible': False}",6,1,0.000000
5,bec96c0b-5b0b-4992-8756-e0d3b704afb5,Ormeasco di Pornassio,5.0,5.000000,"{'actual_k': 1, 'was_impossible': False}",6,1,0.000000
6,bec96c0b-5b0b-4992-8756-e0d3b704afb5,Vermentino Riviera ligure di Ponente,4.0,4.000000,"{'actual_k': 1, 'was_impossible': False}",6,1,0.000000
7,bec96c0b-5b0b-4992-8756-e0d3b704afb5,Aspromonte,5.0,5.000000,"{'actual_k': 1, 'was_impossible': False}",6,1,0.000000
8,bec96c0b-5b0b-4992-8756-e0d3b704afb5,Rossese,3.0,3.000000,"{'actual_k': 1, 'was_impossible': False}",6,1,0.000000
9,e336887b-883b-4d9a-8cd3-060c25856bfb,Tenuta Sant'Helena Pinot Grigio,3.0,3.000000,"{'actual_k': 1, 'was_impossible': False}",3,1,0.000000


In [10]:
knn.get_accuracy(predictions)

RMSE: 0.1792
Precision: 0.80
Recall: 0.78


96.32936507936508

* best predictions

In [11]:
best_pred

,uid,iid,rui,est,details,Iu,Ui,err
0,826c6abe-7147-4b69-aaea-275fe65b033a,Cavalcaonte Barbera Forlí,5.0,5.0,"{'actual_k': 1, 'was_impossible': False}",1,1,0.0
635,09f6ca6e-905f-4afd-bab2-d928fe046f18,Vitovska legno,4.0,4.0,"{'actual_k': 2, 'was_impossible': False}",8,2,0.0
636,09f6ca6e-905f-4afd-bab2-d928fe046f18,Vitovska legno,4.0,4.0,"{'actual_k': 2, 'was_impossible': False}",8,2,0.0
637,09f6ca6e-905f-4afd-bab2-d928fe046f18,Vigna petrussa Rinera,4.0,4.0,"{'actual_k': 1, 'was_impossible': False}",8,1,0.0
642,09f6ca6e-905f-4afd-bab2-d928fe046f18,Refosco dal Peduncolo Rosso Riserva,4.0,4.0,"{'actual_k': 1, 'was_impossible': False}",8,1,0.0
643,8764df8a-0919-433c-beab-053a9c12f402,Nero d'Avola Sicilia,5.0,5.0,"{'actual_k': 1, 'was_impossible': False}",1,2,0.0
644,457b85f5-7b46-4f52-85b9-59acb549b9a2,Limniona Rosé,4.0,4.0,"{'actual_k': 1, 'was_impossible': False}",6,1,0.0
645,457b85f5-7b46-4f52-85b9-59acb549b9a2,Broquel Malbec,4.0,4.0,"{'actual_k': 1, 'was_impossible': False}",6,1,0.0
646,457b85f5-7b46-4f52-85b9-59acb549b9a2,Escapades,4.0,4.0,"{'actual_k': 1, 'was_impossible': False}",6,1,0.0
647,457b85f5-7b46-4f52-85b9-59acb549b9a2,ATMA,3.0,3.0,"{'actual_k': 1, 'was_impossible': False}",6,1,0.0


* worst predictions

In [12]:
worst_pred

,uid,iid,rui,est,details,Iu,Ui,err
698,bc8f3005-c2c6-4277-9fd7-340248f4e7ec,Franciacorta,5.0,4.250000,"{'actual_k': 4, 'was_impossible': False}",11,4,0.750000
697,bc8f3005-c2c6-4277-9fd7-340248f4e7ec,Franciacorta,5.0,4.250000,"{'actual_k': 4, 'was_impossible': False}",11,4,0.750000
492,f8c400a6-ded1-40b8-95d2-a263d05c30ce,Amarone,4.0,3.123807,"{'actual_k': 9, 'was_impossible': False}",159,9,0.876193
587,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Cabernet Franc,4.0,3.000000,"{'actual_k': 5, 'was_impossible': False}",36,7,1.000000
615,ecc462e1-5041-43ca-94c3-c2bfc9b3215e,Cabernet Franc,2.0,3.000000,"{'actual_k': 5, 'was_impossible': False}",36,7,1.000000
696,bc8f3005-c2c6-4277-9fd7-340248f4e7ec,Franciacorta,3.0,4.250000,"{'actual_k': 4, 'was_impossible': False}",11,4,1.250000
955,014e4ed1-6f8b-4b25-917d-c167a2acca17,New Wine,4.0,2.500000,"{'actual_k': 2, 'was_impossible': False}",2,4,1.500000
954,014e4ed1-6f8b-4b25-917d-c167a2acca17,New Wine,1.0,2.500000,"{'actual_k': 2, 'was_impossible': False}",2,4,1.500000
748,4ed273f7-1816-4f36-88c7-789125f011c7,Moscato d'Asti,1.0,3.000000,"{'actual_k': 2, 'was_impossible': False}",4,4,2.000000
747,4ed273f7-1816-4f36-88c7-789125f011c7,Moscato d'Asti,5.0,3.000000,"{'actual_k': 2, 'was_impossible': False}",4,4,2.000000
